https://excelsior-cjh.tistory.com/167

https://github.com/adioshun/gitBook_Machinelearning/blob/master/appendix/kb0809d2b9-c9d5-c0dd-c131-cd94-cd9c+-c120-d0dd.md

PCA는 다음과 같은 단계로 이루어진다. 

- 학습 데이터셋에서 분산이 최대인 축(axis)을 찾는다. 

- 이렇게 찾은 첫번째 축과 직교(orthogonal)하면서 분산이 최대인 두 번째 축을 찾는다.

- 첫 번째 축과 두 번째 축에 직교하고 분산을 최대한 보존하는 세 번째 축을 찾는다.

- 1~3과 같은 방법으로 데이터셋의 차원(특성 수)만큼의 축을 찾는다.


이렇게 i-번째 축을 정의하는 단위 벡터(unit vector)를 i-번째 주성분(PC, Principal Component)이라고 한다.



‌PCA의 목적 : 
- 정보 손실을 최대화 하면서 차원 축소. 즉, 정보 손실을 수량화 하여 최적화 문제 푸는것.
- 정보 손실 = 원래 공간에서 퍼져 있는 정도가 변환(=투영)된 공간에서도 잘 퍼져 있는지 평가 
- 퍼져 있는가? = 분산이 최대 인가 
![](http://i.imgur.com/Ivu5S1I.png)

그림 8.13(a)는 (1,0)으로 변환시 4개의 값이 2개로 줄어듬 
- S_2 = S_1 
- S_4 = S_3 

그림 8.13(b)는 (0,1)으로 변환시 4개의 값이 3개로 줄어듬 
- S_2 = S_2 
- S_4 = S_4
- S_1 = S_3

그림 8.13(c)는 (xxx)으로 변환시 4개의 값이 4개로  유지됨 
- xxx
- xxx

분산 계산 표  



8.13(c)의 분산이 가장 크므로 가장 좋은 축이다. 

즉, 분산을 최대화 하는 축 u를 구하는 문제 임

![](http://i.imgur.com/u9Hc4BN.png)

- 1. (식 8.30)을 보면 u가 크면 분산이 최대화 되므로, u가 무한대이면 정답 
- 2. u는 단위 백터 이므로 uu^t = 1 라는 조건 성립 -> 조건부 최적화 문제 
- 3. 조건부 최적화 문제는 라그랑주 승수를 이용하여 해결 가능 

![](http://i.imgur.com/0DpuOk4.png)

- 4. 라그랑주 승수로 변환된 (식 8.32)을 u로 미분하고 식을 풀어 쓰면 아래와 같이 변함 

![](http://i.imgur.com/999tDvp.png)

> 3번째의 가로안은 공분산 행렬이다. 이를 4번째 줄에서 $\sum(공분산 기호)$로 대치 하였다.

- 5. 라그랑제 상수 L\(u\)를 최대화 하는 U는 $\frac{\partial L(U)}{\partial} =0$을 만족 해야 하므로 , $\frac{\partial \sum\mu - 2\lambda \mu}{\partial} =0$ 이다. 

- 6. 즉, $\sum\mu = 2\lambda \mu$이다. 이를 잘 살펴 보면
	* $\mu$는 공분산 행렬 $\sum$의 고유 벡터\(eigenvector\)
	* $\lambda$는 고유 값\(eigenvalue\)

- 7. 결국 훈련집합의 공분산 행렬 구하기 - 공분산 행렬의 고유 벡터 구하기 - 최대 분산 u 

- 8. 다차원에서 고유값이 클수록 중요도가 크가, 고유값이 큰걸 선택하고 이것의 고유 벡터를 활용
	- 이 고유 백터가 주성분 이다. 



In [1]:
import numpy as np
import numpy.linalg as LA
import pandas as pd
#from sympy import *
np.set_printoptions(precision=4, suppress=True)

## 1. 데이터 생성 

In [2]:
np.random.seed(4)
m = 60
w1, w2 = 0.1, 0.3
noise = 0.1
angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5
X = np.empty((m, 3))
X[:, 0] = np.cos(angles) + np.sin(angles)/2 + noise * np.random.randn(m) / 2
X[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2
X[:, 2] = X[:, 0] * w1 + X[:, 1] * w2 + noise * np.random.randn(m)
print(X)
print('X.shape:', X.shape)


[[-1.0157 -0.5509 -0.2613]
 [-0.0077  0.5996  0.0351]
 [-0.9532 -0.4645 -0.2492]
 [-0.9201  0.2101  0.0218]
 [-0.7631  0.1583  0.1915]
 [ 1.1182  0.3251  0.3171]
 [-1.0226 -0.6438 -0.1337]
 [ 0.6735 -0.2734 -0.0079]
 [ 1.0162  0.5155  0.4678]
 [ 0.5496  0.6773  0.234 ]
 [-0.9896  0.0089 -0.1215]
 [ 1.1325  0.2823  0.0697]
 [-0.9934 -0.2683 -0.1576]
 [-0.9476 -0.4996 -0.1393]
 [ 1.101   0.1706  0.0979]
 [-0.346   0.4533  0.0643]
 [ 0.691  -0.2734 -0.0593]
 [ 0.683   0.7042  0.2464]
 [ 0.8774 -0.1678 -0.0216]
 [-1.0609 -0.4821 -0.3857]
 [ 0.5234  0.6659  0.3902]
 [-0.9442 -0.615  -0.4761]
 [-0.9716  0.0074 -0.1876]
 [-1.1056 -0.319  -0.1719]
 [ 1.1749  0.1572  0.2658]
 [ 0.9134 -0.1608  0.0222]
 [-0.2869  0.5637  0.1121]
 [ 1.1025  0.2102  0.1783]
 [-0.9358  0.1443 -0.0764]
 [ 0.6617  0.7351  0.3744]
 [ 0.099   0.6062  0.1435]
 [-1.081  -0.5469 -0.3707]
 [ 0.03    0.6054  0.1374]
 [ 0.9785 -0.0483  0.093 ]
 [ 0.9726  0.1556  0.2669]
 [-0.0692  0.5703  0.3242]
 [ 0.0169  0.6466  0.1557]
 

## 2. 공분산 행렬 구하기 

In [3]:
X_cen = X - X.mean(axis=0)  # 평균을 0으로
X_cov = np.cov(X_cen.T) #np.dot(X_cen.T, X_cen) / 59  #
print(X_cov)  

[[0.6981 0.1764 0.1214]
 [0.1764 0.1802 0.0725]
 [0.1214 0.0725 0.0455]]


## 3. eigenvalue(w)와 eigenvector(v) 구하기 

- 공분산행렬에서 고유값과 고유벡터를 계산할 수 있다.
- 특성다항식을 이용해 계산하면 되는데, 직관적 이해는 공분산행렬을 선형변환을 통해서 서로 독립인 데이터를 만들수 있다는것이다.
- 그리고 고유벡터는 각각의 벡터가 직교하기 때문에 무엇이든 투영할수 있다.
- 고유값의 크기별로 내림차순으로 되어있기 때문에, 영향력을 말한다고 할수도 있다.
- 따라서 PC1부터 차례대로 기존의 행렬의 내적을 한다면, 축소된 차원이지만 정보는 차있는 알찬 결과를 얻게 된다.



In [4]:
w, v = np.linalg.eig(X_cov)
print('eigenvalue :', w)
print('eigenvector :\n', v)
print('explained variance ratio :', w / w.sum())


eigenvalue : [0.7783 0.1352 0.0103]
eigenvector :
 [[ 0.9364  0.3403 -0.0863]
 [ 0.2985 -0.9012 -0.3142]
 [ 0.1847 -0.2685  0.9454]]
explained variance ratio : [0.8425 0.1463 0.0112]


In [5]:
# sklearn 버젼 
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
pca.fit(X)

PCA(copy=True, iterated_power='auto', n_components=3, random_state=None, svd_solver='auto', tol=0.0, whiten=False)
print('singular value :', pca.singular_values_)
print('singular vector :\n', pca.components_.T)
print('eigen_value :', pca.explained_variance_)
print('explained variance ratio :', pca.explained_variance_ratio_)


singular value : [6.7765 2.824  0.7812]
singular vector :
 [[-0.9364  0.3403 -0.0863]
 [-0.2985 -0.9012 -0.3142]
 [-0.1847 -0.2685  0.9454]]
eigen_value : [0.7783 0.1352 0.0103]
explained variance ratio : [0.8425 0.1463 0.0112]


## 4. Explained Variance Ratio 와 적절한 차원 수 선택하기

각각의 주성분 벡터가 이루는 축에 투영(projection)한 결과의 분산의 비율을 말하며, 각 eigenvalue의 비율과 같은 의미


In [8]:
# PC1에 projection
pc1 = v[:, 0]
proj1 = np.dot(X, pc1)
# PC2에 projection
pc2 = v[:, 1]
proj2 = np.dot(X, pc2)
# PC3에 projection
pc3 = v[:, 2]
proj3 = np.dot(X, pc3)
proj_list = np.array([proj1.var(), proj2.var(), proj3.var()])

print('variance(==eigenvalue) :', proj_list)
print('explained variance ratio :', proj_list / proj_list.sum())





variance(==eigenvalue) : [0.7653 0.1329 0.0102]
explained variance ratio : [0.8425 0.1463 0.0112]


In [6]:
# sklearn 버젼 
print('explained variance ratio :', pca.explained_variance_ratio_)


explained variance ratio : [0.8425 0.1463 0.0112]


각 착으로 투영 하였을때의 정보의 양 
- 첫 번째 주성분 축 : 84.2%
- 두 번째 주성분 축 : 14.6%
- 세 번째 주성분 축 : 1.1%  ( 매우 적은 양의 정보가 들어 있다)

## 5. 적절한 차원 수 선택하기
'Explained Variance Ratio'를 이용해 축소할 차원의 수를 선택할 수 있다. 예를들어, 누적된 분산의 비율이 95%가 되는 주성분 축, 차원을 선택하는 것과 같은 방법이다.



In [9]:
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1
print('선택할 차원 수 :', d)



선택할 차원 수 : 2


In [10]:
# sklearn 버젼 
pca = PCA(n_components=0.95)
X_proj = pca.fit_transform(X)
print('principal component vec :\n', pca.components_.T)



principal component vec :
 [[-0.9364  0.3403]
 [-0.2985 -0.9012]
 [-0.1847 -0.2685]]
